In [1]:
import data_utils as du
import learning_utils as lu

C:\Temp\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ModuleNotFoundError: No module named 'keras'

In [ ]:
data_file = 'kaggle_data/train_data.csv'
labels_file = 'kaggle_data/train_labels.csv'

data, labels = du.Import_Data(data_file, labels_file)

In [ ]:
clean_data = du.Remove_Zero_Variance(data)

In [ ]:
#features = du.Select_Features_From_Model(clean_data, labels)
features = clean_data

In [ ]:
normalized_features = du.Normalize(features, 'min-max')
shuffled_features, shuffled_labels = du.Shuffle(normalized_features, labels)

In [ ]:
resampled_features, resampled_labels = du.Resample(shuffled_features, shuffled_labels, 'InstanceHardnessThreshold')

In [ ]:
train_features, test_features, train_labels, test_labels = du.Split_Data(normalized_features, labels)

In [ ]:
model_svm = lu.Learn_Multiclass_SVM(train_features, train_labels)

In [ ]:
predictions = lu.Predict(model_svm, test_features)
#print(predictions)

In [ ]:
accuracy = lu.Accuracy_Score(test_labels, predictions)

In [ ]:
#model_mlp = lu.Learn_Multilayer_Perceptron(train_features, train_labels, 0.0001)
#mlp_score = lu.Evaluate_NN_Performance(model_mlp, test_features, test_labels)